In [ ]:
!python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py train /u/horlavanasta/MasterProject//DataAndExperiments/Experiments/Experiments-1.5T-3T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmTrue --n_splits 1 --split 0  --batch_size 5               

In [2]:
def check_history(model_path, num_folds):
    from visualize.data_utils import get_data_generic
    from argparse import Namespace
    
    finished_folds=0

#     if not os.path.exists(os.path.join(model_path, "status.txt")):
#         args = Namespace()
#         args.model_path=model_path
#         args.data_types=["history"]
#         args.average_fold=False
#         data = get_data_generic(args, reshape_dict=False)
#         finished_folds=0
#         for fold, fold_data in data["history"].items():
#             if len(fold_data[["epoch"]])==epochs:
#                 finished_folds+=1
#         if finished_folds==num_folds:
#             print("Finished:", model_path)
#             with open(os.path.join(model_path, "status.txt"), "w") as f:
#                 f.write("Status: job was finished for all folds")
#         return finished_folds==num_folds
#     else:
#         return True
    return os.path.exists(os.path.join(model_path, "status.txt"))
    
def check_results(model_path, MS_list, num_folds):
    import os
    import pathlib
    import numpy as np
    currentDirectory = pathlib.Path(model_path)
    currentPattern = "fold-*"
    flag=True
    for fold_dir in currentDirectory.glob(currentPattern):
        fold = int(str(fold_dir).split("-")[-1])

        selection_metrics = ["best_loss", "best_balanced_accuracy", "last_checkpoint"]
        cnn_classification_dir = os.path.join(model_path, 'fold-%i' % fold, 'cnn_classification')
        
        for selection_metric in selection_metrics:
            modes = ['train', 'validation']
            for ms_el in MS_list:
                modes.append('test_' + ms_el)
                
            for mode in modes:
                if not os.path.exists(os.path.join(cnn_classification_dir, selection_metric,
                                                       '%s_image_level_metrics.tsv' % (mode))):
                    flag=False
                
    return flag
    
def check_complete_test(model_path, num_folds, MS_list):
    import json
    path_params = os.path.join(model_path, "commandline_train.json")
#     check_history(model_path, epochs, num_folds)
    return (check_history(model_path, num_folds) and check_results(model_path, MS_list, num_folds))
    


In [3]:
import pathlib
import pandas as pd
import os
import json

folders = []
MS_main_list = ["1.5T-3T", '1.5T', "3T"]
MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'
num_folds_arr=[5]
isBayesian_arr=[True]
num_folds=5
for isBayesian in isBayesian_arr:
    for MS in MS_main_list[:]:
        print("MS %s \n ____________________________________________________________________________________________"%MS)
        model_types = [ "ResNet18", "SEResNet18", "ResNet18Expanded", "SEResNet18Expanded", "Conv5_FC3", "SEResNet50","SEResNet50Expanded", ]
        MS_list = MS_list_dict[MS]
        inference_modes=["mode", "mean"]
        results_folder_general =os.path.join(home_folder, 'Code/ClinicaTools/AD-DL/results/', "Experiments_%d-fold"%num_folds, "Experiments_Bayesian" if isBayesian else "Experiments", 'Experiments-' + MS)
        model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments_%d-fold/Experiments-%s"%(num_folds, MS), "NNs_Bayesian" if isBayesian else "NNs")

        for network in model_types[:]:
            model_dir = os.path.join(model_dir_general, network)
            # output_dir = pathlib.Path(output_dir)
            modelPatter = "subject_model*"
            folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

            for f in folders[:]:
                if check_complete_test(f, num_folds=num_folds,MS_list=MS_list):
#                     pass
                    print(f)
                    for inference_mode in inference_modes:
                        results_dir=os.path.join(results_folder_general, "%s_inference"%inference_mode)
#                         !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py bayesian $f stat
                        !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_distribution results history --get_test_from_bayesian $isBayesian --ba_inference_mode $inference_mode --catplot_type violinplot --uncertainty_metric "total_variance"
#                         !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_distribution results history --average_fold False --get_test_from_bayesian True --ba_inference_mode $inference_mode --catplot_type violinplot --uncertainty_metric "total_variance"

                else: 
                    pass
#                     print(f)

MS 1.5T-3T 
 ____________________________________________________________________________________________
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-1.5T-3T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue_20210604_132156
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-1.5T-3T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmTrue_20210604_132150
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-1.5T-3T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-1.5T-3T/NNs_Bayesian/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmTrue
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments

In [4]:
import pathlib
import pandas as pd
import os
import json

folders = []
MS_main_list = ["1.5T-3T", '1.5T', "3T"]
MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'
num_folds_arr=[5]
isBayesian_arr=[False]
num_folds=5
for isBayesian in isBayesian_arr:
    for MS in MS_main_list[:]:
        print("MS %s \n ____________________________________________________________________________________________"%MS)
        model_types = [ "ResNet18", "SEResNet18", "ResNet18Expanded", "SEResNet18Expanded", "Conv5_FC3", "SEResNet50","SEResNet50Expanded", ]
        MS_list = MS_list_dict[MS]
        inference_modes=["mode", "mean"]
        results_folder_general =os.path.join(home_folder, 'Code/ClinicaTools/AD-DL/results/', "Experiments_%d-fold"%num_folds, "Experiments_Bayesian" if isBayesian else "Experiments", 'Experiments-' + MS)
        model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments_%d-fold/Experiments-%s"%(num_folds, MS), "NNs_Bayesian" if isBayesian else "NNs")

        for network in model_types[:]:
            model_dir = os.path.join(model_dir_general, network)
            # output_dir = pathlib.Path(output_dir)
            modelPatter = "subject_model*"
            folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

            for f in folders[:]:
                if check_complete_test(f, num_folds=num_folds,MS_list=MS_list):
#                     pass
                    print(f)
                    
                    results_dir=os.path.join(results_folder_general, network)
#                         !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py bayesian $f stat
                    !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir results history
#                         !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py visualize $f $results_dir uncertainty_distribution results history --average_fold False --get_test_from_bayesian True --ba_inference_mode $inference_mode --catplot_type violinplot --uncertainty_metric "total_variance"

                else: 
                    pass
#                     print(f)

MS 1.5T-3T 
 ____________________________________________________________________________________________
MS 1.5T 
 ____________________________________________________________________________________________
MS 3T 
 ____________________________________________________________________________________________
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-3T/NNs/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmFalse_20210612_195440
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-3T/NNs/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmTrue_20210612_195437
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experiments-3T/NNs/ResNet18/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmFalse_20210612_195445
/u/horlavanasta/MasterProject/DataAndExperiments/Experiments_5-fold/Experi

In [ ]:
def translate_parameters(args):
    """
    Translate the names of the parameters between command line and source code.
    """
    args.gpu = False
    args.num_workers = args.nproc
    args.optimizer = "Adam"
    args.batch_size=9
    # args.loss = "default"

    if hasattr(args, "caps_dir"):
        args.input_dir = args.caps_dir
    if hasattr(args, "unnormalize"):
        args.minmaxnormalization = not args.unnormalize
    if hasattr(args, "slice_direction"):
        args.mri_plane = args.slice_direction
    if hasattr(args, "network_type"):
        args.mode_task = args.network_type

    if not hasattr(args, "selection_threshold"):
        args.selection_threshold = None
        
    if not hasattr(args, "verbose"):
        args.verbose = 0
    if not hasattr(args, "bayesian"):
        args.bayesian = False

    if not hasattr(args, "prepare_dl"):
        if hasattr(args, "use_extracted_features"):
            args.prepare_dl = args.use_extracted_features
        elif hasattr(args, "use_extracted_patches") and args.mode == "patch":
            args.prepare_dl = args.use_extracted_patches
        elif hasattr(args, "use_extracted_slices") and args.mode == "slice":
            args.prepare_dl = args.use_extracted_slices
        elif hasattr(args, "use_extracted_roi") and args.mode == "roi":
            args.prepare_dl = args.use_extracted_roi

    return args

def show_model(model_folder):
    
    from tools.deep_learning.models import init_model
    from tools.deep_learning.data import (get_transforms,
                                        load_data,
                                        return_dataset,
                                        generate_sampler)
    from tools.deep_learning.iotools import return_logger
    from argparse import Namespace
    from torch.utils.data import DataLoader

    import torchvision.models
    import hiddenlayer as hl
    import torch

    path_params = os.path.join(model_folder, "commandline_train.json")
    with open(path_params, "r") as f:
        params = json.load(f)
    params = translate_parameters(Namespace(**params))
    main_logger = return_logger(params.verbose, "main process")

    model = init_model(params, initial_shape=None)
    from torchsummary import summary
    summary(model, (1, 169, 208, 179))
            
    transforms = [
    hl.transforms.Prune('Constant'),

    # Fold Conv/Relu
    ht.Fold("Conv > Relu", "ConvRelu"),
    # Fold bottleneck blocks
    hl.transforms.Fold("""
        ((ConvRelu > ConvRelu > Conv) | Conv) > Add > Relu
        """, "BottleneckBlock", "Bottleneck Block"),
    # Fold residual blocks
    hl.transforms.Fold("""ConvRelu > ConvRelu > Conv > Add > Relu""",
                       "ResBlock", "Residual Block"),

    # Fold Conv, BN, RELU layers into one
    hl.transforms.Fold("Conv > BatchNorm > Relu", "ConvBnRelu"),
    # Fold Conv, BN layers together
    hl.transforms.Fold("Conv > BatchNorm", "ConvBn"),
    # Fold bottleneck blocks
    hl.transforms.Fold("""
        ((ConvBnRelu > ConvBnRelu > ConvBn) | ConvBn) > Add > Relu
        """, "BottleneckBlock", "Bottleneck Block"),
    # Fold residual blocks
    hl.transforms.Fold("""ConvBnRelu > ConvBnRelu > ConvBn > Add > Relu""",
                       "ResBlock", "Residual Block"),
    # Fold repeated blocks
    hl.transforms.FoldDuplicates(),
]
    hl_graph=hl.build_graph(model, torch.zeros([1, 1, 169, 208, 179]), transforms=transforms)
    hl_graph.theme = hl.graph.THEMES["blue"].copy()  # Two options: basic and blue    
    hl_graph.save('/u/horlavanasta/MasterProject/cnn_hiddenlayer_%s'%params.model, format='png')
    

def show_fpg(data_batch, indices=None,plane="sag", num_rows=2, 
    num_cols=2, name=None, folder="/current_augmentations_examples/"):
    
    import matplotlib.pyplot as plt
    import numpy as np
    
    fig, axes = plt.subplots(num_rows, num_cols, figsize=((int(8 * num_rows), int(6 * num_cols))))
    data_batch=data_batch.numpy()
    print(data_batch.shape)
    data_batch=data_batch[:num_rows*num_cols].reshape(num_rows, num_cols, data_batch.shape[1],  data_batch.shape[2],  data_batch.shape[3], 
                                  data_batch.shape[4])
    print(data_batch.shape)

    for row in range(num_rows):
        for col in range(num_cols):

            i, j, k = indices
            data=data_batch[row][col]
            kwargs = dict(cmap='gray', interpolation='none')
            slices=dict()
            slices["sag"], slices["cor"], slices["axi"] = np.rot90(data[0, i]), np.rot90(data[0, :, j]), np.rot90(data[0, ..., k])

            axes[row][col].imshow(slices[plane],**kwargs)
            axes[row][col].axis('off')
            
#     path = '../../outputs/'+folder
#     if not os.path.exists(path):
#         os.makedirs(path)
    if name is not None:
        fig.suptitle(name)
    plt.subplots_adjust( left=0.05, right=0.95, top=0.95, bottom=0.05, wspace=0.05, hspace=0.05)

#     plt.savefig(path + str(name) + '.png')
    plt.show()
    plt.close()
    
def show_data(model_folder, name=None, plane="sag"):
    from tools.deep_learning.models import init_model
    from tools.deep_learning.data import (get_transforms,
                                        load_data,
                                        return_dataset,
                                        generate_sampler)
    from tools.deep_learning.iotools import return_logger
    from argparse import Namespace
    from torch.utils.data import DataLoader

    import torchvision.models
    import hiddenlayer as hl
    import torch

    path_params = os.path.join(model_folder, "commandline_train.json")
    with open(path_params, "r") as f:
        params = json.load(f)
    params = translate_parameters(Namespace(**params))
    main_logger = return_logger(params.verbose, "main process")

    
    train_transforms, all_transforms = get_transforms(params.mode,
                                                      minmaxnormalization=params.minmaxnormalization,
                                                      data_augmentation=None,
                                                      output_dir=None)
    training_df, valid_df = load_data(
            params.tsv_path,
            params.diagnoses,
            0,
            n_splits=params.n_splits,
            baseline=params.baseline,
            logger=main_logger
        )

    
    data_valid = return_dataset(params.mode, params.input_dir, valid_df, params.preprocessing,
                                train_transformations=train_transforms, all_transformations=all_transforms,
                                params=params)

    
    valid_loader = DataLoader(
        data_valid,
        batch_size=params.batch_size,
        shuffle=False,
        num_workers=params.num_workers,
        pin_memory=True
    )
    
    sample = next(iter(valid_loader))
    
    show_fpg(sample["image"], indices=(169//2, 208//2, 179//2), name=name, plane=plane)
    
    

In [ ]:
import pathlib
import pandas as pd
import os
import json

folders = []
MS_main_list = ['1.5T', "3T","1.5T-3T" ]
MS_list_dict = {'1.5T':['1.5T', '3T'], "3T": ['3T', '1.5T'], "1.5T-3T": ["1.5T-3T"]}
home_folder='/u/horlavanasta/MasterProject/'

isBayesian=True
for MS in MS_main_list[:1]:
    print("MS %s \n ____________________________________________________________________________________________"%MS)
    model_types = [ "ResNet18", "SEResNet18", "ResNet18Expanded", "SEResNet18Expanded", "Conv5_FC3" ]
    MS_list = MS_list_dict[MS]
    
    model_dir_general = os.path.join(home_folder,"DataAndExperiments/Experiments/Experiments-" + MS, "NNs" if isBayesian else "NNs")

    for network in model_types[:]:
        model_dir = os.path.join(model_dir_general, network)
        # output_dir = pathlib.Path(output_dir)
        modelPatter = "subject_model*"
        folders = [f for f in pathlib.Path(model_dir).glob(modelPatter)]

        for f in folders[:1]:
            print(f)
            show_model(f)
#             show_data(f, plane="sag")
#             show_data(f, plane="cor")
#             show_data(f, plane="axi")
             

In [ ]:
hl_graph